<a href="https://colab.research.google.com/github/denny-erikson/wagnanimus-code/blob/main/Wagnanimu's_Code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Wagnanimu's Code** 🛠️

In [ ]:
# @markdown ---

# @title Passo 1 - Preparação 🛠️
# @markdown 🔗 **Montando Diretórios:** Configurando o ambiente para acesso aos diretórios 📂
# @markdown - 🔒 **Solicitando Permissão de Acesso** ao Google Drive para carregar e salvar arquivos 🔐
# @markdown - ✅ **Aguarde** a execução completa desta etapa antes de prosseguir ⏳
# @markdown ---
# @markdown 🚨 **Importante:** Por favor, **não abra nem edite este código** para evitar erros no processo! ⚠️


from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# @title Passo 2 - Processamento XML ⚡
# @markdown ⚡ **Execução da Regra de Processamento do XML:** Aplicando as regras definidas no arquivo XML
# @markdown - ✅ **Aguarde** a execução completa desta etapa ⏳
# @markdown ---
# @markdown 🚨 **Importante:** Para garantir a integridade do processamento, **não abra nem edite este código!** ⚠️


import xml.etree.ElementTree as ET
import os
import shutil
from datetime import datetime
import pandas as pd
import re

# Dicionário para mapeamento de months em português
months = {
    '01': 'JANEIRO', '02': 'FEVEREIRO', '03': 'MARÇO',
    '04': 'ABRIL', '05': 'MAIO', '06': 'JUNHO',
    '07': 'JULHO', '08': 'AGOSTO', '09': 'SETEMBRO',
    '10': 'OUTUBRO', '11': 'NOVEMBRO', '12': 'DEZEMBRO'
}

# Dicionário para mapeamento de filiais com base no CNPJ
cnpj_branches = {
    '62438841000132': "NR SAM",
    '62438841000647': "NR SAP",
    '62438841000485': "NR SP"
}

# Função para criar o caminho baseado no XML
def extract_and_create_path(xml_path, base_url):
    tree = ET.parse(xml_path)
    root = tree.getroot()

    def find_cpf_or_cnpj(path):
      return root.find(f'{path}CPF').text if root.find(f'{path}CPF') is not None else root.find(f'{path}CNPJ').text if root.find(f'{path}CNPJ') is not None else None


    cnpj_receiver = root.find('.//{http://www.portalfiscal.inf.br/nfe}dest/{http://www.portalfiscal.inf.br/nfe}CNPJ').text
    cnpj_provider = find_cpf_or_cnpj('.//{http://www.portalfiscal.inf.br/nfe}emit/{http://www.portalfiscal.inf.br/nfe}')
    corporate_reason = root.find('.//{http://www.portalfiscal.inf.br/nfe}emit/{http://www.portalfiscal.inf.br/nfe}xNome').text
    invoice_number = root.find('.//{http://www.portalfiscal.inf.br/nfe}ide/{http://www.portalfiscal.inf.br/nfe}nNF').text
    issue_date = root.find('.//{http://www.portalfiscal.inf.br/nfe}ide/{http://www.portalfiscal.inf.br/nfe}dhEmi').text

    corporate_reason = re.sub(r'[^a-zA-Z0-9]', ' ', corporate_reason)

    issue_date = datetime.fromisoformat(issue_date)
    month_in_portuguese = months[issue_date.strftime("%m")]
    formatted_date = f"{issue_date.strftime('%Y')}/{month_in_portuguese}/{issue_date.strftime('%d')}"

    branch = cnpj_branches.get(cnpj_receiver[:14], "Outra Filial")
    folder_name = f"NF {invoice_number} - {corporate_reason}"
    file_name = f"NF {invoice_number} - {corporate_reason}.xml"
    full_path = os.path.join(base_url, branch, formatted_date, folder_name, file_name)
    payload = {
        "cnpj_receiver": cnpj_receiver,
        "cnpj_provider": cnpj_provider,
        "corporate_reason": corporate_reason,
        "invoice_number": invoice_number,
        "issue_date": formatted_date,
        "branch": branch
    }

    return full_path, invoice_number, corporate_reason, payload

# Função para salvar as métricas em uma planilha Excel
def salvar_metricas(metrics, path):
    df = pd.DataFrame([metrics])

    if os.path.exists(path):
        existing_df = pd.read_excel(path)
        df = pd.concat([existing_df, df], ignore_index=True)

    df.to_excel(path, index=False)

# Função para processar arquivos XML e PDFs
def processar_arquivos_na_pasta(pasta, base_url, user):
    metrics = {
        "Usuário": user,
        "Data de Execução": datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
        "Tempo de Execução": None,
        "Quantidade de arquivos": 0,
        "Filial": "",
        "Data": "",
        "Número NFS-e": "",
        "Nome/Razão Social": "",
        "CNPJ": "",
        "Caminho Salvo": "",
    }

    start_time = datetime.now()  # Início da contagem do tempo

    for arquivo in os.listdir(pasta):
        if arquivo.endswith('.xml'):
            xml_path = os.path.join(pasta, arquivo)
            full_path, invoice_number, corporate_reason, payload = extract_and_create_path(xml_path, base_url)

            try:
                os.makedirs(os.path.dirname(full_path), exist_ok=True)
                if os.path.exists(full_path):
                    print(f"Conflito: o arquivo {full_path} já existe. Arquivo não movido.")
                    continue
                shutil.move(xml_path, full_path)
                print(f"Arquivo XML {arquivo} movido e renomeado com sucesso!")

                metrics["Quantidade de arquivos"] += 1
                metrics["Filial"] += f'{payload["branch"]}, '
                metrics["Data"] += f'{payload["issue_date"]}, '
                metrics["Número NFS-e"] += f'{payload["invoice_number"]}, '
                metrics["Nome/Razão Social"] += f'{payload["corporate_reason"]}, '
                metrics["CNPJ"] += f'{payload["cnpj_provider"]}, '
                metrics["Caminho Salvo"] += f'{full_path}, '

            except Exception as e:
                print(f"Erro ao mover o arquivo XML {arquivo}: {e}")
                continue

            # Procurar e mover o PDF correspondente
            for pdf_file in os.listdir(pasta):
                if pdf_file.endswith('.pdf') and invoice_number in pdf_file:
                    pdf_path = os.path.join(pasta, pdf_file)
                    pdf_new_name = full_path.replace('.xml', '.pdf')

                    try:
                        if os.path.exists(pdf_new_name):
                            print(f"Conflito: o arquivo PDF {pdf_new_name} já existe. Arquivo não movido.")
                            continue
                        shutil.move(pdf_path, pdf_new_name)
                        print(f"Arquivo PDF {pdf_file} movido e renomeado com sucesso!")
                    except Exception as e:
                        print(f"Erro ao mover o arquivo PDF {pdf_file}: {e}")
                    break

    metrics["Tempo de Execução"] = str((datetime.now() - start_time).total_seconds())

    excel_path = "/content/drive/Shareddrives/dalilas_code_homologacao/produtos/data/metricas.xlsx"
    salvar_metricas(metrics, excel_path)
    print("Métricas salvas com sucesso!")


In [ ]:
import os
# @title Wagnanimu's Code 🌐
# @markdown #### 🌍 *Selecione o Ambiente de Trabalho*
# @markdown ##### 🔄 Etapas para garantir o sucesso do processamento:
# @markdown - ✅ **Espere** a execução das etapas anteriores ⏳
# @markdown - ✅ **Preencha** os campos para garantir a integridade do processamento ✏️
# @markdown - ✅ **Carregue** ou certifique-se de ter carregado os dados em [UPLOADS 🔗](https://drive.google.com/drive/folders/1wvXNEzpzmErngQ_Ezv92-BvHJfS6YZSt?usp=drive_link)
# @markdown - ✅ **Acompanhe** os logs ao término da execução 📋
# @markdown - ✅ **Processo finalizado com sucesso!** 🎉
# @markdown - 📂 Para acessar os resultados, clique aqui: [PRODUTOS 🔗](https://drive.google.com/drive/folders/1btHxpjtvOPoR6TbaYCUCk-gQDB2zqx87?usp=sharing)
# @markdown - 📊 **Dica**: Confira os logs para garantir que tudo está conforme esperado.
# @markdown - 👊🏽 **Obrigado por utilizar o Wagnanimu's Code** 🚀


# @markdown ---

#   @markdown #### *Selecione o Ambiente
enviroment_path = "PRODUCTION" # @param ["HOMOLOGATION","PRODUCTION"]
# @markdown # <br/>
# @markdown #### *Selecione o Usuário
user = "aprendiz.financeiro" # @param ["denny.pimenta","wagner.almeida","jhonatan.sansao","mayara.santos", "juliana.oliveira","cassia.oliveira","aprendiz.financeiro"]

drive_directory = "/content/drive/Shareddrives"
enviroments = {
    "HOMOLOGATION": "dalilas_code_homologacao",
    "PRODUCTION": "Financeiro"
}
enviroment = f"{drive_directory}/{enviroments[enviroment_path]}"

if(os.path.exists(enviroment)):
  base_directory = f"{enviroment}/Notas Fiscais Entrada/Nota Fiscal Produto/"
  file_path = os.path.join(drive_directory, enviroments["HOMOLOGATION"], "produtos/uploads")
  save_directory = os.path.join(base_directory)
  print(save_directory)


  if os.path.exists(file_path) and any(os.listdir(file_path)):
    processar_arquivos_na_pasta(file_path, save_directory, user)
else:
    print("Pasta ou arquivos não encontrados")

# @markdown ---

/content/drive/Shareddrives/Financeiro/Notas Fiscais Entrada/Nota Fiscal Produto/
Conflito: o arquivo /content/drive/Shareddrives/Financeiro/Notas Fiscais Entrada/Nota Fiscal Produto/NR SAP/2024/OUTUBRO/11/NF 5822 - SANTOS   BORGES TINTAS LTDA/NF 5822 - SANTOS   BORGES TINTAS LTDA.xml já existe. Arquivo não movido.
Arquivo XML 35241009507184000155550010000113791543242915.xml movido e renomeado com sucesso!
Arquivo PDF 35241009507184000155550010000113791543242915.pdf movido e renomeado com sucesso!
Métricas salvas com sucesso!
